In [1]:
Debug = False   # if debug mode.  Debug = True. if Release mode, Debug = False.
DebugPageLen = 1
DebugItemLen = 6
infoHead = ("id","status","start","end","title","consultPrice","marketPrice","currentPrice","bidCount","delayCount","applyCount")

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd
import re

In [3]:
# time
def getCurrentTimeStamp():
    t = time.time()
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t)))
    return t


def printDuration(start, end):
    seconds = end - start
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    print ("%02d:%02d:%02d" % (h, m, s))
    
    

# import time
# dt = "2019-06-03 10:00:00"
# #转换成时间数组
# timeArray = time.strptime(dt, "%Y-%m-%d %H:%M:%S")
# #转换成时间戳
# timestamp = time.mktime(timeArray)
# print (timestamp)

In [4]:
class DataManagement:
    def __init__(self):
        self.data = {}
        self.data["name"] = None

        self.cvsFileName = 'namelist.csv'
        self.xlsxFileName = 'shanghai2019.xlsx'
        
        self.__openFile()

    def __processName(self,attribute, data):
        if(attribute == "name"):
            index = data.find("通过")
            data = data[4:index]
            self.data[attribute] =  data     
            
    def __processArea(self,attribute, data):
        left = data.find("建筑面积")
        if(left != -1):
            size = data[(left+4) : (left+22)]
            areas = re.findall(r"\d+\.\d*",size)
            self.data[attribute] = areas[0] if areas else 0
        else:
            self.data[attribute] = 0
        
    def __processDate(self,attribute, data):
        time1 = int(int(data)/1000 )
        self.data[attribute] = str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time1)))

    def __processCommon(self,attribute, data):
        self.data[attribute] =  data        
        
        


    def setData(self,attribute, data):
        if(attribute == "name"):
            self.__processName(attribute, data)
        elif(attribute == "areaA" or attribute == "areaB"):
            self.__processArea(attribute, data)
        elif(attribute == "start" or attribute == "end"):
            self.__processDate(attribute, data)
        else:
            self.__processCommon(attribute,data)

    
    def __clacData(self):
        # 
        if (int(self.data["consultPrice"]) > 0):
            originPrice = self.data["consultPrice"]
        else:
            originPrice = self.data["marketPrice"]
        #
        if (int(originPrice)  == 0):
            self.data["discount"] = "error"
        else:
            self.data["discount"]  = float("%.2f" % float(int(self.data["currentPrice"])/originPrice))
 
       # change unit to from RMB yuan to  RMB Wan
        self.data["originPrice"]  = float("%.2f" % float(int(originPrice)/10000))        
        self.data["currentPrice"] = float("%.2f" % float(int(self.data["currentPrice"])/10000))

      # set area
        if(float(self.data["areaA"]) > 0):
            self.data["area"]  = self.data["areaA"]
            self.data["unit"] = float("%.2f" % float(int(self.data["currentPrice"])/float(self.data["area"]) ))
        elif(float(self.data["areaB"]) > 0):
            self.data["area"] = self.data["areaB"]
            self.data["unit"] = float("%.2f" % float(int(self.data["currentPrice"])/float(self.data["area"]) ))
        else:
            self.data["area"] = 0
            self.data["unit"] = 0
      ###    
        
    
    def getData(self,attribute):
        return self.data[attribute]
    
    def __openFile(self):
        self.csvFile = open(self.cvsFileName,"w+")
        self.writer = csv.writer(self.csvFile)        
        header = ("name","start","end","title","originPrice","currentPrice","discount","unit","bidCount","delayCount","applyCount","area","areaA","areaB","url")
        self.writer.writerow(header)

    def writeFile(self):
        
        self.__clacData()
        
        
        self.writer.writerow((
                              self.data["name"],
                              self.data["start"],
                              self.data["end"],
                              self.data["title"],
                              self.data["originPrice"],
                              self.data["currentPrice"],
                              self.data["discount"],
                              self.data["unit"],
                              self.data["bidCount"],
                              self.data["delayCount"],
                              self.data["applyCount"],
                              self.data["area"],
                              self.data["areaA"],
                              self.data["areaB"],
                              self.data["url"]
                           ))

    
    def closeFile(self):
        self.csvFile.close()
        csv = pd.read_csv(self.cvsFileName, encoding='utf-8')
        csv.to_excel(self.xlsxFileName, sheet_name='data')
            


In [5]:
# time
startTime =  getCurrentTimeStamp()

2019-06-01 12:57:25


In [6]:
data = DataManagement()


In [7]:
# tianjin
# sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
#                  "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  


# wuhan
# sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&city=%CE%E4%BA%BA&sorder=2" \
#                   "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-19&spm=a213w.3064813.9001.2"

#shanghai
# sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%C9%CF%BA%A3&sorder=2" \
#                  "&st_param=-1&auction_start_from=2019-01-01&auction_start_to=2019-05-24&spm=a213w.3064813.9001.2"

# ningbo todo 
# sourceUrl = "https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.83.9c5216a9elw0w9&category=50025969&auction_source=0" \
#                  "&city=%C4%FE%B2%A8&sorder=1&st_param=-1&auction_start_seg=-1"

#tianjin 2017-2018
sourceUrl =  "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
                  "&st_param=-1&auction_start_from=2017-01-01&auction_start_to=2018-01-01&spm=a213w.3064813.9001.2"

In [8]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [9]:
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))


<selenium.webdriver.remote.webelement.WebElement (session="251fa18082cbf8d4d68ca030b7590dc0", element="0.06258808490833978-1")>

In [10]:
# TEST

if(Debug == True):
    pages = DebugPageLen
else:
    pages = driver.find_element_by_class_name("page-total").text


In [11]:
for page in range(1,int(pages)+1):
    if(page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
    jsonData =  json.loads(driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML'))
    items = jsonData["data"]
    
    if(Debug ==True):
        itemLen = DebugItemLen
    else:
        itemLen = len(items)


    for i in range(itemLen):            
        if(items[i]["status"] == "done" or items[i]["status"] == "todo"):            
            for key,value in items[i].items():
                if key in infoHead:
                    data.setData(key,value)

            url = "#pai-item-"+str(data.getData("id"))
                        
            # Click the link which opens in a new window
            driver.find_element_by_css_selector(url).click()
            time.sleep(1) # have to wait sometime

            # Wait for the new window or tab
            wait.until(EC.number_of_windows_to_be(2))
            handles = driver.window_handles
            driver.switch_to.window(handles[1])
            # size
            
            urlNew = driver.current_url
            data.setData("url",urlNew)

            
            if(data.getData("status") == "done"):
                driver.get(str(urlNew)+"#J_Confirmation")
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))            
                data.setData("name",driver.find_element_by_class_name("c-content").text)
#                 data.setData("date",driver.find_element_by_class_name("c-stamp").text)

           # get area
            driver.get(str(urlNew)+"#NoticeDetail")
            time.sleep(1)
            data.setData("areaA",driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent'))

           ###  Tip
            driver.get(str(urlNew)+"#RemindTip")
            time.sleep(1)
            data.setData("areaB",driver.find_element_by_id("J_desc").get_attribute('textContent'))
                

### write into file
            data.writeFile()
          
            driver.close()
            driver.switch_to.window(handles[0])
        else:
            pass
           # print("***item failed******"+": Page :"+str(page)+"   Item :"+str(item))



TimeoutException: Message: 


In [ ]:
driver.quit()

In [ ]:
data.closeFile()

In [ ]:
endTime = getCurrentTimeStamp()

printDuration(startTime, endTime)